## Setup

In [1]:
from Crypto.Util.number import bytes_to_long
from hashlib import shake_128
from secrets import token_bytes
from math import floor, ceil, log2
import os, enum

m = 256
w = 21
n = 128
l1 = ceil(m / log2(w))
l2 = floor(log2(l1*(w-1)) / log2(w)) + 1
l = l1 + l2

print(f"m={m}, w={w}, n={n}, l1={l1}, l2={l2}, l={l}")

class HashType(enum.IntEnum):
    MSG = 0
    WOTS_PK = 1
    WOTS_CHAIN = 2
    TREE_NODE = 3

def F(data: bytes, seed: bytes, length: int, type: int) -> bytes:
    hasher = shake_128(seed + bytes([type]) + data)
    return hasher.digest(length)

class WOTS:
    def __init__(self, seed: bytes):
        self.seed = seed
        self.sk = [token_bytes(n // 8) for _ in range(l)]
        self.pk = [WOTS.chain(sk, w - 1, seed) for sk in self.sk]
    
    def sign(self, digest: bytes) -> bytes: # list
        assert 8 * len(digest) == m
        d1 = WOTS.pack(bytes_to_long(digest), l1, w)
        checksum = sum(w-1-i for i in d1)
        d2 = WOTS.pack(checksum, l2, w)
        d = d1 + d2

        sig = [WOTS.chain(self.sk[i], w - d[i] - 1, self.seed) for i in range(l)]
        return sig

    def get_pubkey_hash(self) -> bytes:
        return F(b"".join(self.pk), self.seed, 16, HashType.WOTS_PK)

    @staticmethod
    def pack(num: int, length: int, base: int) -> list[int]: # 小端序转w进制数的列表
        packed = []
        while num > 0:
            packed.append(num % base)
            num //= base
        if len(packed) < length:
            packed += [0] * (length - len(packed))
        return packed
    
    @staticmethod
    def chain(x: bytes, n: int, seed: bytes) -> bytes:
        if n == 0:
            return x
        x = F(x, seed, 16, HashType.WOTS_CHAIN)
        return WOTS.chain(x, n - 1, seed)

    @staticmethod
    def verify(digest: bytes, sig: bytes, seed: bytes) -> bytes:
        d1 = WOTS.pack(bytes_to_long(digest), l1, w)
        checksum = sum(w-1-i for i in d1)
        d2 = WOTS.pack(checksum, l2, w)
        d = d1 + d2

        sig_pk = [WOTS.chain(sig[i], d[i], seed) for i in range(l)]
        return F(b"".join(sig_pk), seed, 16, HashType.WOTS_PK)

class MerkleTree:
    def __init__(self, height: int, seed: bytes):
        self.h = height
        self.seed = seed
        self.keys = [WOTS(seed) for _ in range(2**height)]
        self.tree = []
        self.root = self.build_tree([key.get_pubkey_hash() for key in self.keys])
    
    def build_tree(self, leaves: list[bytes]) -> bytes:
        self.tree.append(leaves)

        if len(leaves) == 1:
            return leaves[0]
        
        parents = []
        for i in range(0, len(leaves), 2):
            left = leaves[i]
            if i + 1 < len(leaves):
                right = leaves[i + 1]
            else:
                right = leaves[i]
            hasher = F(left + right, self.seed, 16, HashType.TREE_NODE)
            parents.append(hasher)
        
        return self.build_tree(parents)

    def sign(self, index: int, msg: bytes) -> list:
        digest = F(msg, self.seed, 32, HashType.MSG)
        key = self.keys[index]
        wots_sig = key.sign(digest)
        sig = [wots_sig]
        for i in range(self.h):
            leaves = self.tree[i]
            u = index >> i
            if u % 2 == 0:
                if u + 1 < len(leaves):
                    sig.append((0, leaves[u + 1])) # 兄弟节点
                else:
                    sig.append((0, leaves[u]))
            else:
                sig.append((1, leaves[u - 1]))
        return sig
    
    @staticmethod
    def verify(root, sig: list, msg: bytes, seed: bytes) -> bytes:
        digest = F(msg, seed, 32, HashType.MSG)
        wots_sig = sig[0]
        sig = sig[1:]
        pk_hash = WOTS.verify(digest, wots_sig, seed)
        computed_root = pk_hash
        for (side, leaf) in sig:
            if side == 0:
                computed_root = F(computed_root + leaf, seed, 16, HashType.TREE_NODE)
            else:
                computed_root = F(leaf + computed_root, seed, 16, HashType.TREE_NODE)
        return root == computed_root

def serialize_signature(sig) -> bytes:
    data = b"".join(sig[0])
    for side, node in sig[1:]:
        data += bytes([side]) + node
    return data

def deserialize_signature(data: bytes):
    sig = []
    sig.append([data[i*16:(i+1)*16] for i in range(l)])
    data = data[l*16:]
    height = (len(data)) // 17
    for i in range(height):
        side = data[i*17]
        node = data[i*17+1:(i+1)*17]
        sig.append((side, node))
    return sig


m=256, w=21, n=128, l1=59, l2=3, l=62


## working

In [ ]:
from tqdm import tqdm
import time
from pwn import *

token = b""

for i in tqdm(range(100)):
    time.sleep(2)
    r = remote('prob18.geekgame.pku.edu.cn', 10018)
    r.recv()

    r.sendline(token)
    info = r.recv()
    print(info)

    match = re.search(r"Seed: ([a-f0-9]+)", info.decode())
    if match:
        SEED = bytes.fromhex(match.group(1))
        print(f"提取到的seed: {SEED}")

    digest = F(b"Give me the flag", SEED, 32, HashType.MSG)
    d1 = WOTS.pack(bytes_to_long(digest), l1, w)
    checksum = sum(w-1-i for i in d1) # 平均 20*59/2 = 590
    print(checksum)
    if checksum > 680:
        print(f"Seed: {SEED}")
        break

In [4]:
def get_d(msg):
    digest = F(msg, SEED, 32, HashType.MSG)
    d1 = WOTS.pack(bytes_to_long(digest), l1, w)
    checksum = sum(w-1-i for i in d1)
    # print(checksum)
    d2 = WOTS.pack(checksum, l2, w)
    newd = d1 + d2
    return newd

# SEED = bytes.fromhex("81b8ce61255e1a0e47e27bdf914e1dbc")
print(SEED, SEED.hex(), len(SEED))
flag_d = get_d(b"Give me the flag")

b'\x81\xb8\xcea%^\x1a\x0eG\xe2{\xdf\x91N\x1d\xbc' 81b8ce61255e1a0e47e27bdf914e1dbc 16
684


In [5]:
import random
import string
from tqdm import tqdm
maxlen, besti, best = 0, "", []

for i in tqdm(range(2**18)):
    bytes_i = "".join([random.choice(string.ascii_letters) for _ in range(32)])
    #print(bytes_i)
    newd = get_d(bytes_i.encode())
    tmpl = len([i for i in range(l) if newd[i] >= flag_d[i]])

    if tmpl > maxlen:
        maxlen = tmpl
        besti = bytes_i
        best = [i for i in range(l) if newd[i] >= flag_d[i]]

print(maxlen, besti, best)

100%|██████████| 262144/262144 [00:10<00:00, 24056.89it/s]

51 jjIhQNXUsqMvZdMhzSCITfRmXaAdJNWy [0, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 55, 56, 58, 59, 60]


In [6]:
bestmatch = 0
for i in tqdm(range(2**20)):
    bytes_i = "".join([random.choice(string.ascii_letters) for _ in range(42)])
    # print(bytes_i)
    newd = bytes_i.encode(bytes_i.encode())
    tmpl = len([i for i in range(l) if newd[i] >= flag_d[i] or i in best])
    bestmatch = max(bestmatch, tmpl)
    if tmpl == l:
        print(bytes_i)
        break

print(bestmatch)

  4%|▎         | 37946/1048576 [00:02<00:53, 18867.44it/s]

ZXxKFrRdozhyNoXptYQxDwWPbGzUHtfcPZfIsKWlUH
62


## local test

In [7]:
tree = MerkleTree(8, SEED)

msg1 = b"jjIhQNXUsqMvZdMhzSCITfRmXaAdJNWy"
msg2 = b"ZXxKFrRdozhyNoXptYQxDwWPbGzUHtfcPZfIsKWlUH"

sig1 = tree.sign(-1, msg1)
sig2 = tree.sign(255, msg2)

assert sig1[1:] == sig2[1:]
wots_sig1 = sig1[0]
wots_sig2 = sig2[0]

In [8]:
target_d1 = get_d(msg1)
target_d2 = get_d(msg2)

wots_flag = []
for i in range(62):
    if target_d1[i] >= flag_d[i]:
        tmp = WOTS.chain(wots_sig1[i], target_d1[i] -flag_d[i], SEED)
    elif target_d2[i] >= flag_d[i]:
        tmp = WOTS.chain(wots_sig2[i], target_d2[i] -flag_d[i], SEED)
    else:
        print("???")
        break
    wots_flag.append(tmp)

sig_flag = [wots_flag] + sig1[1:]

In [9]:
MerkleTree.verify(tree.root, sig_flag, b"Give me the flag", SEED)

True

## online

In [10]:
msg1 = b"jjIhQNXUsqMvZdMhzSCITfRmXaAdJNWy"
msg2 = b"ZXxKFrRdozhyNoXptYQxDwWPbGzUHtfcPZfIsKWlUH"

r.sendline(b"1")
print(r.recv())
r.sendline(b"-1")
print(r.recv())
r.sendline(msg1)
print(r.recvuntil(b"2) Request flag\n> "))

r.sendline(b"1")
print(r.recv())
r.sendline(b"255")
print(r.recv())
r.sendline(msg2)
print(r.recvuntil(b"2) Request flag\n> "))

b'Index: '
b'Message: '
b'a7c4c3fd887e7bddf48a529f0cacd1afb7a45db2dc9d41d6c0ab26a34a4ea158e3b037105e627fc783ee199ec2a7e359484ee7e172accee594964f25272cd42694f13a2d35dc6bec7aca48c7d7fd11f3bf79ff206614e5e416724c7a8f480ecbace09f1940f04ae87144146c6fbe8ba653e7f2ad050a2b1b26308989ed16ccd30cad43c2f8e77c4b2584ef6484a5dcfc4cdd13240761258f0bc6c24805878d672636e08f625a5cd65598fb23761400b92f437b27ba8f2e718c77bf6f7ccad665b5352d0d7ef3eac053a1ea335977fb10c7559adfda87f34780342133b62602c12c42fa050f833fe32f96ca852f96f0cf453e439f6dd8d1c8cf77ad062bf51837aa788c9c178f055b2c207d1871a394ba9acc4b8b3d5ab388bb73a0a0e15c0749850152859dcbe88f37a0217e44252b713748122e0405c3fddb3c623455a6c91e6a0df9566186c2a2ad7f2f3c7bb3cb35bff1bedf457c052d0a7affe3761124d47c0c1c2117976d3eea289fb6b27c86bf3ae4430ea1290f4e6331308ff1dba30688e86abd428ed0fb7718532eb3a36ad4109671883023821a809810cc5325975ac36367407196b627e3601fc1001884c0b437ada80748ddba83a55c0439b9444ed7af52edfa4e2e5d2621b20bfd78e4aa41d2035ee3eeedcd7c22bc9a656eca0afb6965e7584b58

In [11]:
sig1 = deserialize_signature(bytes.fromhex("a7c4c3fd887e7bddf48a529f0cacd1afb7a45db2dc9d41d6c0ab26a34a4ea158e3b037105e627fc783ee199ec2a7e359484ee7e172accee594964f25272cd42694f13a2d35dc6bec7aca48c7d7fd11f3bf79ff206614e5e416724c7a8f480ecbace09f1940f04ae87144146c6fbe8ba653e7f2ad050a2b1b26308989ed16ccd30cad43c2f8e77c4b2584ef6484a5dcfc4cdd13240761258f0bc6c24805878d672636e08f625a5cd65598fb23761400b92f437b27ba8f2e718c77bf6f7ccad665b5352d0d7ef3eac053a1ea335977fb10c7559adfda87f34780342133b62602c12c42fa050f833fe32f96ca852f96f0cf453e439f6dd8d1c8cf77ad062bf51837aa788c9c178f055b2c207d1871a394ba9acc4b8b3d5ab388bb73a0a0e15c0749850152859dcbe88f37a0217e44252b713748122e0405c3fddb3c623455a6c91e6a0df9566186c2a2ad7f2f3c7bb3cb35bff1bedf457c052d0a7affe3761124d47c0c1c2117976d3eea289fb6b27c86bf3ae4430ea1290f4e6331308ff1dba30688e86abd428ed0fb7718532eb3a36ad4109671883023821a809810cc5325975ac36367407196b627e3601fc1001884c0b437ada80748ddba83a55c0439b9444ed7af52edfa4e2e5d2621b20bfd78e4aa41d2035ee3eeedcd7c22bc9a656eca0afb6965e7584b58aac2f17e6861281662cd2a51715edaa28d02a971d64038a397d5b21b96587d93dda3df422f79d45ecf44bdd1a9e0d0bd0f03aa28f9b98769399d15ccb58cd8e7f4b0f6af4cce0805d3829f4b7e0933458e1ec4489a509eab20209c012321788af67e1c033de5af5b18cd1a489b5ba34d4fd4a89710a8fe15f8f9dc888b5252e5840d916e315abcc0740b0af2ffcfbf130a8603ef9168f96c627235ea26f7c907a08f500a5e651c59a4208199d24f32e437a69c1ca93ddca6390e51232af0c8146ca20db676fc2c85ebf1b83615c62e9b389e9a0cb956a276506215f6391959bac6ea41e0ecfb161510ea6ec3cd6459e379f90e704fefa53325f26125a65bcdc1b04226978ef6f1daab9e7ecf5955eb9846deb07fceeeceecda456c02051048a2f1c96acc7527989d95965d525b71d7026de56661919e676f60d15907b2bde8ec0050ffb057278df3e6a6df52476201eda8fb55f8fdcf643673b47679d0081117d5a7732a6f74b081485afd577d9d90b94d3ebba041704914fba7fb0f509bf797792988c093727c6b207fe12aab85c94aceb506967f6549d3e22c953b3f782c1356cde31ac028563b643163d47ebef0dd201d7b19d58a47cba87fa8ed85ef6bdbed8c4a675520c26e44cf036796303a23482f5cb187cd4c8aada00520057db2cf93c1c87155243766e8c92a111dae42698649219eac84b63d5d011145217052a79c40e8619d2425ab93ac01fa15fdf9743e69f4f7deec1b20c8277801c876078e69efddfd55af116e547eb1d5018e086ae78407640d64d20f2070f4aa6f0115b27f7ace7c468a511be6089aab08c601b03734d5fbc204b159839a63ab20b64201dc4af68d9f29cf1a2e6949aa18f851300198a07de8a29a22f3b0fe11bf8dacd109"))
sig2 = deserialize_signature(bytes.fromhex("969d51f3033c1e0138a8aae2a9c6f5b4553d569dbfdf6f62f07cffddad10a92c53c091d9b991aedcf2f82208e05a65d78f7aff200ff589ff80abb8b61142064f8c80ba36caa0d20e6bc7617423ed601166775407b44326ad4e10e738f909786ac19c0d6adda037d852fbe617345b7a878397b7a716626fa63739de4ed3d6e8f5aea5ddef0a720278cd17344f0dbcad78bbae29a0db7cf708c2d66db8d8041daa0142a9e7071af07c54a3c917fe1754f3e686e5f4e834920aec2ab21df4e8f65ab5352d0d7ef3eac053a1ea335977fb10780cf7672b19cb66f1b5e91195102331ae5ef69c6a80cba2e39e0290c307e3cd638b092bea43d1a2eac81111677fa1ad2383b683c3c780a104b2c4517528e3d4846825ffe5b06c3402b5869616e3dec5dd7a49d78c388bcb72a8900fddc72e304f1840b091408ef5d11d525428bfacbf0c25d6a76102cdd7e5149264bf65441741e3799573fdbfb8677798f2e1b979d6feab0b5edd26f4b978351d013757cfc190922becff594257ea1f6c0e3aa2a37388e86abd428ed0fb7718532eb3a36ad4a88b0aba66812131f6ba63d9e7ccf055eb1c34f10593eb5e98814c0f306b93e095de1d3a04e515eca72b655e0cff3c734a7c67847d0724b1998e4722a129c2958ab4d81cc419945c768de5c93e0f29f5341f1cb1eed8f5a3e0729779fae980df2f0d68924ee70a0183abe01af28f801ca2051318b76584964aecf8d9d82f9a533a217fce8a8f095659fe0187b150dd0eec2585b1a073bcb1a85b50d0a56d72715bd7dbd61460200438e4636f98623ed1c12031979e769fdc28889419eae3fc4c19d663bdabd29ebb6a5c4402eb6b4d7547c756e1b74553cdffda183581ecbbf743c1f7e0dc991b19f239dbf376db67499b42390dd61198f8ca45883908aa92f2f706d542e68cd61649a41b65dc25ca6edf1e26124cdadb44d7938c47360f7b01c291393f0ec2ec1471c1551037dac77357ddf2524134a4704476f2a101e821f45b96cac7b2361572d6ae07fd05a907fb8db83acc7c5679dc865f62d5dfe108a82b9d46e3e24cb72dcb83106204db184b1c2df014ee827893bcaad83e891adf8f5be32ff66b22999ced8c5bcb621b66a3d15907b2bde8ec0050ffb057278df3e6a6df52476201eda8fb55f8fdcf64367363596693be627c40171e470e1d1b67382fdd132946dcfbfc4c4fe4d8e31d2bbbad9e5c82667143a8042b503303e7ce7f7fe12aab85c94aceb506967f6549d3e2350a13de8fd1439b067c51ad989be7d1abdebf7d386b4096126c4fbcd939d0b980c1fdcc1276d9716b4a2895a7a967bfcf036796303a23482f5cb187cd4c8aadf4f4f1094bf803d8fb3d10341783efe28d368fd19988f864b9e2fa0613f41073011145217052a79c40e8619d2425ab93ac01fa15fdf9743e69f4f7deec1b20c8277801c876078e69efddfd55af116e547eb1d5018e086ae78407640d64d20f2070f4aa6f0115b27f7ace7c468a511be6089aab08c601b03734d5fbc204b159839a63ab20b64201dc4af68d9f29cf1a2e6949aa18f851300198a07de8a29a22f3b0fe11bf8dacd109"))

In [12]:
assert sig1[1:] == sig2[1:]
wots_sig1 = sig1[0]
wots_sig2 = sig2[0]

In [13]:
target_d1 = get_d(msg1)
target_d2 = get_d(msg2)

In [14]:
wots_flag = []
for i in range(62):
    if target_d1[i] >= flag_d[i]:
        tmp = WOTS.chain(wots_sig1[i], target_d1[i] - flag_d[i], SEED)
    elif target_d2[i] >= flag_d[i]:
        tmp = WOTS.chain(wots_sig2[i], target_d2[i] - flag_d[i], SEED)
    else:
        print("???")
        break
    wots_flag.append(tmp)

In [15]:
sig_flag = [wots_flag] + sig1[1:]

In [16]:
print(serialize_signature(sig_flag).hex())

bc3f893ef173c32ad7bf7cfe9382519ff2742a9164573f1e89f5949bfc78528c63e02c9873576cec88bcba3cf8d03939c9775cf286a5e3a233a7d4cc2864c35026bc12f1522b1d19037af63d32aef35d56d841a5bce1ffd4fd012951b3e976f6c19c0d6adda037d852fbe617345b7a87f4553e88f99eaf97bd48d5a72a96f180167bbc1ce122a27dd7a152a08867eb854491387b9b0df4cc805763179b358ff22239559d543f2d00dbe2946f9e45dbbddb5c8e976515e15a3837bfa8faa357487c545de9fb7abbed48323ab75a5677dc47c0b4ed242ce90b214ea7a5010e582876205bfcb9a4e89e6ae810e96efef1184fd382e2fb85905b89d489fb4edd928eb05a3d8641587e541a13d64d7d67cfd2846825ffe5b06c3402b5869616e3dec5850152859dcbe88f37a0217e44252b71888d523d7d302042372feee3791512faa1c5f29e84f383956fe64b1ca7f995ec8a0536b286d358a03259abbcfc4c85e80f1499cfe4cf84b784538bd96cf8c68ae8aae8e79c3974da4d0e27444d960c82d1aaaeef72e083c9d7cfd357dd39fedc5905787232efb8cf220d65b4ed4078be7fc414e81c6c85fd51805e98848c166b5d3b24cc6e58a63f32409888085f39395045dec59893d4161a2ce4e5db1c8b52da25515536f6a11f681cf429933dc74a2d4b8384d102c079240b36359cf765a153c2657c

In [18]:
r.sendline(b"2")
print(r.recv())
r.sendline(b"bc3f893ef173c32ad7bf7cfe9382519ff2742a9164573f1e89f5949bfc78528c63e02c9873576cec88bcba3cf8d03939c9775cf286a5e3a233a7d4cc2864c35026bc12f1522b1d19037af63d32aef35d56d841a5bce1ffd4fd012951b3e976f6c19c0d6adda037d852fbe617345b7a87f4553e88f99eaf97bd48d5a72a96f180167bbc1ce122a27dd7a152a08867eb854491387b9b0df4cc805763179b358ff22239559d543f2d00dbe2946f9e45dbbddb5c8e976515e15a3837bfa8faa357487c545de9fb7abbed48323ab75a5677dc47c0b4ed242ce90b214ea7a5010e582876205bfcb9a4e89e6ae810e96efef1184fd382e2fb85905b89d489fb4edd928eb05a3d8641587e541a13d64d7d67cfd2846825ffe5b06c3402b5869616e3dec5850152859dcbe88f37a0217e44252b71888d523d7d302042372feee3791512faa1c5f29e84f383956fe64b1ca7f995ec8a0536b286d358a03259abbcfc4c85e80f1499cfe4cf84b784538bd96cf8c68ae8aae8e79c3974da4d0e27444d960c82d1aaaeef72e083c9d7cfd357dd39fedc5905787232efb8cf220d65b4ed4078be7fc414e81c6c85fd51805e98848c166b5d3b24cc6e58a63f32409888085f39395045dec59893d4161a2ce4e5db1c8b52da25515536f6a11f681cf429933dc74a2d4b8384d102c079240b36359cf765a153c2657cf7ce2e17c3de1281c4e5e090a2051318b76584964aecf8d9d82f9a53153f5baa172e9373eaa0b44073228c7072fd70db3d7add7ebc6609ea49b6228c09e49f4394034f4505029a433d1e7db4461eff1e778ee97c48a0c43a4e2e6eb32574c796beb41af16c4bb005f97eb81fb8264d8661b6f1b4e4add3ebeb569286022042db03c284e234c3cf3a326888c17679f2be2bc95c45a86f4738c3d3ca32441836a7e9f887b7952018e0f8c89e63df1e26124cdadb44d7938c47360f7b01d873216300ec2a4ace4658b6db64985a8089d51d922f816198ce0216c6a8cd6d5b96cac7b2361572d6ae07fd05a907fb393a56ce8a4af915ba35bc49f56197859d23fc4ff78570a72e83d86379a2f715e2741a6d7c382eeb344db3987fc90f5f965d525b71d7026de56661919e676f60d15907b2bde8ec0050ffb057278df3e6f4019bdd2fe466e87e301842ecdc219463596693be627c40171e470e1d1b6738faafc80d2135fc950eacad3bfa1111761089fdab132399e2d7dd673b262f510246c1029dcb77d907684a437d6a8212801e4457d32a4e771bfde8fc10642ec3d0f73f622a2e506bc58ee34b7ba918e7577fa8ed85ef6bdbed8c4a675520c26e44c265f2cd488f4fbd3042144e421206dd9a5413466fe6e35e49bf39ef1739ab788d368fd19988f864b9e2fa0613f41073011145217052a79c40e8619d2425ab93ac01fa15fdf9743e69f4f7deec1b20c8277801c876078e69efddfd55af116e547eb1d5018e086ae78407640d64d20f2070f4aa6f0115b27f7ace7c468a511be6089aab08c601b03734d5fbc204b159839a63ab20b64201dc4af68d9f29cf1a2e6949aa18f851300198a07de8a29a22f3b0fe11bf8dacd109")
print(r.recv())

b'Congratulations! Here is your flag: flag{N3ga7IVe_1ndEX_RU1NEd_mY_5cHeME!!!}\nChoose an action:\n1) Sign message\n2) Request flag\n> '
